In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window as W
from pyspark.sql.functions import row_number
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from datetime import datetime

# O objetivo desta tabela é identificar a última compra realizada por cada cliente, informando em qual shopping ocorreu a compra e a respectiva localidade.

df_sales_silver = spark.table('b_shopping.sales')\
    .select('customer_id', 'category', 'invoice_date', 'shopping_mall')\
    .withColumn('invoice_date', F.to_date(F.col('invoice_date'), 'MM/dd/yyyy'))\
    .withColumn('shopping',  F.when(F.col('shopping_mall') == 'Irvine Spectrum', 'Irvine Spectrum Center').otherwise(F.col('shopping_mall'))) # No df_sales_silver o nome do Shopping está escrito diferente do df_shopping_silver se essa tratativa não ser feita isso irá gerar dados nulos no join.

window_spec = W.partitionBy("customer_id").orderBy("invoice_date")

df_window = df_sales_silver.withColumn("row_number", row_number().over(window_spec))\
    .filter(F.col('row_number') == '1')\
    .select('customer_id', 'category', 'invoice_date', 'shopping')

df_customer_silver = spark.table('b_shopping.customer')\
    .select('customer_id', 'gender', 'age')\
    .withColumn('age', F.col('age').cast('int'))

df_shopping_silver = spark.table('b_shopping.shopping_mall')\
    .select('shopping_mall', 'location')

df_final = df_window.join(df_customer_silver, on='customer_id', how='left')\
    .join(df_shopping_silver, df_shopping_silver.shopping_mall == df_sales_silver.shopping, how='left')\
    .select('customer_id', 'gender', 'age', 'invoice_date', 'category','shopping_mall', 'location')

df_final.write.format('delta').mode('overwrite').saveAsTable("s_shopping.TABE_ULTI_COMP")

# Como se trata de uma tabela pequena para evitar Overhead de Gerenciamento a tabela não foi particionada

In [0]:
%sql
SELECT * FROM s_shopping.TABE_ULTI_COMP LIMIT 100

customer_id,gender,age,invoice_date,category,shopping_mall,location
C106444,Male,48,2021-01-03,Toys,Irvine Spectrum Center,Irvine
C107163,Female,57,2021-01-03,Food & Beverage,Westfield Valley Fair,Santa Clara
C109509,Female,60,2021-01-03,Cosmetics,Stanford Shopping Center,Palo Alto
C110454,Female,49,2021-01-03,Clothing,South Coast Plaza,Costa Mesa
C114958,Male,51,2021-01-03,Souvenir,Westfield Century City,Los Angeles
C121997,Male,38,2021-01-03,Cosmetics,The Grove,Los Angeles
C124977,Female,22,2021-01-03,Cosmetics,Fashion Valley,San Diego
C130016,Female,51,2021-01-03,Cosmetics,Westfield Century City,Los Angeles
C133554,Male,49,2021-01-03,Shoes,Beverly Center,Los Angeles
C142209,Female,29,2021-01-03,Food & Beverage,South Coast Plaza,Costa Mesa


In [0]:
from pyspark.sql.functions import date_format

# O objetivo desta tabela é consolidar os dados de vendas, agrupando-os por shopping, categoria e período (cd_peri, representando ano e mês). Embora a visão gerada tenha características que poderiam enquadrá-la na camada Gold, optei por mantê-la na camada Silver para preservar maior flexibilidade e aderência ao processo de transformação dos dados.

df_sales_agg = spark.table('b_shopping.sales')\
    .select('shopping_mall', 'price', 'category', 'invoice_date')\
    .withColumn('invoice_date', F.to_date(F.col('invoice_date'), 'MM/dd/yyyy'))\
    .withColumn('cd_peri', F.date_format('invoice_date', 'yyyyMM').cast('int'))\
    .withColumn('price', F.col('price').cast(DecimalType(18,2)))\
    .groupBy('shopping_mall', 'category','cd_peri').agg(F.sum('price').alias('TOTAL')).orderBy('shopping_mall', 'category', 'cd_peri')

df_sales_agg.write.format('delta')\
    .mode('overwrite')\
    .saveAsTable('s_shopping.TABE_VEND_AGRUP_CATEG')

# Como se trata de uma tabela pequena para evitar Overhead de Gerenciamento a tabela não foi particionada 

In [0]:
%sql
SELECT * FROM s_shopping.TABE_VEND_AGRUP_CATEG LIMIT 100

shopping_mall,category,cd_peri,TOTAL
Beverly Center,Books,202102,469.65
Beverly Center,Clothing,202102,51013.60
Beverly Center,Cosmetics,202102,3862.70
Beverly Center,Food & Beverage,202102,345.18
Beverly Center,Shoes,202102,37210.54
Beverly Center,Souvenir,202102,293.25
Beverly Center,Technology,202102,21000.00
Beverly Center,Toys,202102,2150.40
Del Amo Fashion Center,Books,202102,1424.10
Del Amo Fashion Center,Clothing,202102,216957.84
